This notebook exports data from the tensorboard for several metrics/losses simultaneously.
The code is based on:
https://stackoverflow.com/questions/72837772/download-all-of-csv-files-of-tensorboard-at-once

In [ ]:
import pandas as pd
import requests
from csv import reader
import os

In [ ]:
def URLs(version, split):
    URLs_dict = {
    f'{split}_total_loss' : f'http://localhost:6006/experiment/defaultExperimentId/data/plugin/scalars/scalars?tag={split}_total_loss&run=version_{version}&format=csv',
    }
    for metric in ['disc', 'gen', 'kl', 'nll', 'perc_img', 'rec_img', 'rec_lat']:
        URLs_dict[f'{split}/{metric}_loss'] = f'http://localhost:6006/experiment/defaultExperimentId/data/plugin/scalars/scalars?tag={split}%2F{metric}_loss&run=version_{version}&format=csv'
    return URLs_dict

In [ ]:
urls_dict = URLs(9, 'val')
dfs = {}

for key, url in urls_dict.items():
    try:
        r = requests.get(url, allow_redirects=True)
        data = r.text
        data_csv = reader(data.splitlines())
        data_csv = list(data_csv)
        df = pd.DataFrame(data_csv)
        headers = df.iloc[0]
        df = pd.DataFrame(df.values[1:], columns=headers)
        dfs[key] = df
    except Exception as e:
        print(f"Error fetching {key}: {e}")

# Merge dataframes
merged_df = pd.DataFrame()
for key, df in dfs.items():
    # rename 'Value' column to key
    df.rename(columns={'Value': key}, inplace=True)
    # drop 'Wall time' column
    df.drop(columns=['Wall time'], inplace=True)
    if merged_df.empty:
        merged_df = df
    else:
        merged_df = pd.merge(merged_df, df, on='Step', how='outer')

In [ ]:
def tb_data(version, split):
    urls_dict = URLs(version, split)
    dfs = {}
    
    for key, url in urls_dict.items():
        try:
            r = requests.get(url, allow_redirects=True)
            data = r.text
            data_csv = reader(data.splitlines())
            data_csv = list(data_csv)
            df = pd.DataFrame(data_csv)
            headers = df.iloc[0]
            df = pd.DataFrame(df.values[1:], columns=headers)
            dfs[key] = df
        except Exception as e:
            print(f"Error fetching {key}: {e}")

    # Merge dataframes
    merged_df = pd.DataFrame()
    for key, df in dfs.items():
        # rename 'Value' column to key
        df.rename(columns={'Value': key}, inplace=True)
        # drop 'Wall time' column
        df.drop(columns=['Wall time'], inplace=True)
        if merged_df.empty:
            merged_df = df
        else:
            merged_df = pd.merge(merged_df, df, on='Step', how='outer')
    
    # Sort by 'Step' (int)
    merged_df['Step'] = merged_df['Step'].astype(int)
    merged_df.sort_values(by='Step', inplace=True)
    merged_df.reset_index(drop=True, inplace=True)
    
    return merged_df

In [ ]:
df = tb_data(9, 'val')

In [ ]:
df

In [ ]:
# export df to csv
df.to_csv('tb_data.csv', index=False)